<a href="https://colab.research.google.com/github/Yuns-u/Naver_Series_Reviews_Analysis/blob/main/%ED%95%9C%EA%B5%AD%EC%96%B4_text%EC%8B%9C%EB%8F%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 필요한 모듈 불러오기

In [ ]:
pip install konlpy

In [ ]:
pip install scikit-learn

In [ ]:
from konlpy.tag import Kkma
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np
from math import log # IDF 계산을 위해
import matplotlib.pyplot as plt


# 전처리

텍스트를 문장으로 이루어진 리스트로 만들고 불용어 등을 처리해주기

## 정규표현식으로 전처리하기

----

In [ ]:
#데이터 불러오기
from google.colab import files
uploaded = files.upload()

import pandas as pd
df = pd.read_csv('iOS_train.csv')

In [ ]:
#불필요한 column 지우기
df = df.drop(columns='Unnamed: 0')
df.head()

In [ ]:
import re 

#한글만 남기고 특수기호를 정규표현식을 데이터프레임에 적용해서 모두 없앰.
#df['REVIEW']=df['REVIEW'].str.replace(pat=r'[^\w]', repl= r' ', regex=True)
#df

In [ ]:
def cleasing(text): 
  repl ='' 
  pattern = '([ㄱ-ㅎㅏ-ㅣ]+)' # 자음, 모음 제거 
  text = re.sub(pattern= pattern, repl=repl, string=text) 
  #pattern = '[^\w\s]' # 특수기호 제거 
  pattern = '[^가-히\s]' # 특수기호 제거 
  text = re.sub(pattern= pattern, repl=repl, string=text) 
  pattern = '<[^>]*>' # html 제거 
  text = re.sub(pattern = pattern, repl='',string=text) 
  return text


In [ ]:
df['REVIEW'] = df['REVIEW'].map(lambda x: cleasing(x))
df

# 토큰화

한국어 토큰화를 도와주는 형태소 분석기는 konlpy의 Kkma, Okt, mecab 또는 Pykomoran 등이 있고 성능이 각각 다르다고 한다.

- 1) morphs : 형태소 추출
- 2) pos : 품사 태깅(Part-of-speech tagging)
- 3) nouns : 명사 추출


In [ ]:
from konlpy.tag import Okt  
okt = Okt()

print(okt.morphs("아이패드 앱에서 다크모드 및 스플릿뷰는 언제 지원될까요?"))
print(okt.pos("아이패드 앱에서 다크모드 및 스플릿뷰는 언제 지원될까요?"))
print(okt.nouns("아이패드 앱에서 다크모드 및 스플릿뷰는 언제 지원될까요?"))

In [ ]:
kkma = Kkma()

print(kkma.morphs("아이패드 앱에서 다크모드 및 스플릿뷰는 언제 지원될까요?"))
print(kkma.pos("아이패드 앱에서 다크모드 및 스플릿뷰는 언제 지원될까요?"))
print(kkma.nouns("아이패드 앱에서 다크모드 및 스플릿뷰는 언제 지원될까요?"))

키워드를 추출할 때에 많은 경우 명사를 조합하여 간략하게 표현했었다.
명사를 추출하는 것만 해도 본 프로젝트의 목표를 달성하는 데에는 충분할 것 같다.
또한 kkma에서는 외래어에 한국어가 들어가면 일단 한국어로 의미가 되는 부분을 나눈다는 것을 확인할 수 있었다. 어떤 모듈을 쓰느냐에 따라 성능의 차이가 클 것이라 생각된다.

In [ ]:
okt.nouns("웹툰이랑 소설 같이 볼 수 있어서 좋구 디자인도 깔끔해서 보기 좋아요!\n아쉬운 점은 네이버 웹툰처럼 캡쳐할 수 있는 기능 달아주셨으면 좋겠어요ㅠㅠㅠ")

In [ ]:
kkma.nouns('다신 안 쓸거임! 고마워요😆!!')

In [ ]:
okt.nouns('다신 안 쓸거임! 고마워요😆!!')

In [ ]:
okt.pos('다신 안 쓸거임! 고마워요😆')

하지만 부정적인 평가의 경우 위처럼 명사만 추출할 시(사실 명사라고 보기 애매하다..) 명사형 어간어미도 명사라고 보는 등
정확도도 떨어지고 문맥을 알 수 없어 오히려 난감할 수 있다. 따라서 조사나 어말어미 등은 나중에 불용어처리로 지워주는 것이 좋을 것이라 생각한다.

In [ ]:
# 불용어 정의
stopwords=['의','가','이','은','들','는','좀','잘','걍','과','도','를','을','으로','자','에','와','한','하다','요', '아니',
           '해서','에서','제','신건','아니','저기','주세요','해','서','되는','논','여','저','로','으로','오','고','랑','이랑',
           '합니다','부터','\n','\n\n','왜','입니다','에도','하고','만','다','너무','했는데','수','것','때','거','하는','있는',
           '하는']

In [ ]:
tokenizer = Okt()

tokenized=[]
for sentence in df['REVIEW']:
    temp = tokenizer.nouns(sentence) # 토큰화morphs를 쓸 때보다 명사만 추출했을 때 더 잘 파악됨. 한국어 표제어 추출이 어려움.
    temp = [word for word in temp if not word in stopwords] # 불용어 제거
    tokenized.append(temp)

In [ ]:
print(tokenized[:10])

In [ ]:
df['tokens'] = tokenized
df['tokens']

In [ ]:
# 리뷰 길이 분포 확인
print('리뷰의 최대 길이 :',max(len(l) for l in tokenized))
print('리뷰의 평균 길이 :',sum(map(len, tokenized))/len(tokenized))
plt.hist([len(s) for s in tokenized], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
from collections import Counter

word_counted = Counter()
df['tokens'].apply(lambda x: word_counted.update(x))

#빈도 상위 토큰 보기
#빈도를 보면서 불용어에 지속적으로 업데이트함.
word_counted.most_common(30)

In [ ]:
#토큰화된 문서들을 입력받아 토큰을 센 뒤 관련된 속성을 가진 데이터프레임을 return
def word_count(docs):
  #전체 코퍼스에서 단어 빈도 카운트
  word_counts = Counter()

  #단어가 존재하는 문서의 빈도 카운트, 단어가 한 번 이상 존재하면 1을 더한다.
  word_in_docs = Counter()

  #전체 문서의 개수
  total_docs = len(docs)

  for doc in docs:
    word_counts.update(doc)
    word_in_docs.update(set(doc))

  temp = zip(word_counts.keys(), word_counts.values())
  wc = pd.DataFrame(temp, columns=['word','count'])

  #단어 순위
  #method='first':같은 값의 경우 먼저 나온 요소를 우선
  wc['rank'] = wc['count'].rank(method='first', ascending=False)
  total = wc['count'].sum()

  #코퍼스 내 단어의 비율
  wc['percent'] = wc['count'].apply(lambda x: x/ total)
  wc = wc.sort_values(by='rank')

  #누적 비율
  #cumsum() : cumulative sum
  wc['cul_percent'] = wc['percent'].cumsum()

  temp2 = zip(word_in_docs.keys(), word_in_docs.values())
  ac = pd.DataFrame(temp2, columns=['word','word_in_docs'])
  wc = ac.merge(wc, on='word')

  #전체 문서 중 존재하는 비율
  wc['word_in_docs_percent'] = wc['word_in_docs'].apply(lambda x: x/total_docs)

  return wc.sort_values(by='rank')

In [ ]:
wc = word_count(df['tokens'])
wc.head(20)

In [ ]:
wc.describe()

In [ ]:
#토큰 순위에 따른 퍼센트 누적 분포 그래프
import seaborn as sns

sns.lineplot(x='rank', y='cul_percent', data=wc)

# Lemmetization 표제어 추출해보기


In [ ]:
lem_df = df.copy()
lem_df
#한국어 표제어 추출은 일단 보류해야할 것 같다...

# tf-idf

In [ ]:
tfodf_vector = TfidfVectorizer(min_df=1)
tfidf_matrix = tfodf_vector.fit_transform(df['tokens'])

doc_distance = (tfidf_matrix*tfidf_matrix.T)


# 키워드 추출
KR-WordRank
비지도학습 방법으로 한국어 텍스트에서 단어/키워드를 자동으로 추출하는 라이브러리

In [ ]:
pip install krwordrank

In [ ]:
from krwordrank.word import KRWordRank

min_count = 3   # 단어의 최소 출현 빈도수 (그래프 생성 시)
max_length = 10 # 단어의 최대 길이
wordrank_extractor = KRWordRank(min_count=min_count, max_length=max_length)

In [ ]:
beta = 0.85    # PageRank의 decaying factor beta
max_iter = 10
texts = lem_df['REVIEW'].values
keywords, rank, graph = wordrank_extractor.extract(texts, beta, max_iter)

Graph ranking 이 높은 노드들(substrings)이 후처리 과정을 거쳐 단어로 출력.

키워드 (단어) 추출을 한 결과는 아래와 같다.

In [ ]:
for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:30]:
        print('%8s:\t%.4f' % (word, r))

# Word2Vec

단어를 벡터로 만드는 방법으로 가장 널리 사용되는 임베딩 방법.
- CBoW: 주변 단어에 대한 정보를 기반으로 중심 단어의 정보를 예측하는 모델
- Skip-gram: 중심 단어의 정보를 기반으로 주변 단어의 정보를 예측하는 모델*

**Skip-gram**

- 입력: 원핫인코딩된 단어벡터
- 은닉: 임베딩 벡터의 차원수만큼의 노드로 구성된 은닉층이 1개인 신경망
- 출력: 단어 개수만큼의 노드로 이루어져 있으며 활성화 함수로 소프트맥스 사용

In [ ]:
pip install gensim --upgrade

In [ ]:
from gensim.models.word2vec import Word2Vec
from tqdm import tqdm

In [ ]:
model = Word2Vec(sentences = tokenized, window = 5, min_count = 3, workers = 4, sg = 0)

In [ ]:
# 완성된 임베딩 매트릭스의 크기 확인
#493개의 단어이며 각 단어는 100차원으로 구성되어 있음.
model.wv.vectors.shape

In [ ]:
print(model.wv.most_similar("시리즈"))

In [ ]:
print(model.wv.most_similar("쿠키"))

In [ ]:
print(model.wv.most_similar("무료"))

문서 내 각 단어들을 Word2Vec을 통해 단어 벡터로 변환하고, 이들의 평균으로 문서 벡터를 구하여 선호하는 키워드와 유사한 키워드를 찾아주는 리뷰 키워드 추천 시스템

# fastText 


In [ ]:
pip install fasttext

In [ ]:
from pprint import pprint as print
from gensim.models import fasttext
#from gensim.test.utils import datapath
from gensim import models

ko_model = models.fasttext.load_facebook_model('/Users/yunsu/Downloads/wiki.ko/wiki.ko.vec')

for w, sim in m_fasttext.similar_by_word('파이썬', 10):
    print(f'{w}: {sim}')